# Text to Mesh

### Import modules and set up device

In [ ]:
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image

# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


### Create __[Stable Diffusion pipeline](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/text2img)__ using diffusers and generate an image

In [ ]:
from diffusers import StableDiffusionPipeline

IMG_SIZE = 512  # @param {type:"integer"}

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "interior of a mountain chalet by frank lloyd wright"
image = pipe(prompt, height=IMG_SIZE, width=IMG_SIZE, num_inference_steps=30).images[0]
plt.imshow(image)

### Create __[DPT Depth Estimation pipeline](https://huggingface.co/docs/transformers/main/en/model_doc/dpt#transformers.DPTForDepthEstimation)__ and generate depth map image

In [ ]:
from transformers import pipeline
from PIL import Image
pipe = pipeline("depth-estimation", model="Intel/dpt-large")
result = pipe(image)
depth_img = result["depth"]


### Invert depth values and construct a rgbd image

In [ ]:
import open3d as o3d

width, height = image.size

depths = np.array(depth_img)
depths = depths.max() - depths

# create rgbd image
depth_o3d = o3d.geometry.Image((depths  ).astype('uint8'))
image_o3d = o3d.geometry.Image(np.array(image))
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)



### Create a camera and use to construct a point cloud from the rgbd image

In [ ]:
# camera settings
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width/2, height/2)

# create point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)

# estimate normals
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction((0, 0, -1))

# remove points that are too close to the camera
min_distance_from_camera = 0.0002
points = np.asarray(pcd.points)
ind = np.where(points[:,2] > min_distance_from_camera)[0]
pcd = pcd.select_by_index(ind)

# visualize the point cloud
o3d.visualization.draw_geometries(
    [pcd],
    zoom=0.5,
    front=pcd.get_center() - [0, 0, 0.1],
    lookat=pcd.get_center(),
    up=[0, -1, 0]
)



### Use __[Poisson surface reconstruction](http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Poisson-surface-reconstruction)__ to convert point cloud to mesh

In [ ]:
# surface reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, n_threads=1)
vertices_to_remove = densities < np.quantile(densities, 0.05)
mesh.remove_vertices_by_mask(vertices_to_remove)


o3d.visualization.draw_geometries(
    [mesh], zoom=0.5,
    front=pcd.get_center() - [0, 0, 1],
    lookat=pcd.get_center(),
    up=[0, -1, 0],
    mesh_show_back_face=True,
)



### Save mesh

In [ ]:
import time
basename = "_".join(prompt.split(" ")[:10]) + f"-{int(time.time())}"
o3d.io.write_triangle_mesh(f"{basename}.ply",
                               mesh,
                               write_triangle_uvs=True)

image.save(f"images/{basename}.png")